# Single Agent Dynamics

Rust (1987)

*Optimal Replacement of GMC Bus Engines: An Empirical Model of Harold Zurcher*

## Setting

- An agent owns a fleed to buses
- Buses get old over time
- The older the bus is, the most costly it is to mantain
- The agent can decide to replace the bus engine with a new one, at a cost
- **Dynamic trade-off**
    - What is the best moment to replace the engine?
    - You don't want to replace an engine too early
        - doesn't change much
    - You don't want to replace an engine too late
        - avoid unnecessary maintenance costs

- **State**: mileage of the bus

    $$x_t \in \{1, ..., 10\}$$
    
- **State transitions**: with probability $\lambda$ the mileage of the bus increases
    
    $$
    x_{t+1} = \begin{cases}
    \min \left\{ x_t + 1,10 \right\}  & \text { with probability } \lambda \\ 
    x_t & \text { with probability } 1 - \lambda
    \end{cases}
    $$
    
    Note that $\lambda$ does not depend on the value of the state

- **Action**: replacement decision

    $$
    i_t \in \{0, 1 \}
    $$
    
- **Payoffs**

 - Per-period maintenance cost 
 - Cost of replacement
    
    $$
    u\left(x_{t}, i_{t}, \epsilon_{1 t}, \epsilon_{2 t} ; \theta\right)= 
    \begin{cases}
    -\theta_{1} x_{t}-\theta_{2} x_{t}^{2}+\epsilon_{0 t}, & \text { if } i_{t}=0 \\ 
    -\theta_{3}+\epsilon_{1 t}, & \text { if } i_{t}=1
    \end{cases}
    $$

# Simulation

- Start with an initial value function $V(x_t)=0$
- Compute expected value w.r.t. $\lambda$
  
  $$
  W(x_t) = \begin{cases}
  -\theta_1 x_t - \theta_2 x_t^2 + \beta \Big[(1-\lambda) V(x_t) + \lambda V(\min \{x_t+1,10 \}) \Big] , & \text { if } i_t=0 \\
  -\theta_3 + \beta \Big[(1-\lambda) V(0) + \lambda V(1) \Big] , & \text { if } i_t=1
\end{cases}
  $$
  
- Compute the new value of V

  $$
  V'(x_t) = \log \Big( e^{W(x_t|i_t=0)} + e^{W(x_t|i_t=1)} \Big)
  $$
  
- Repeat until convergence

In [532]:
# Set parameters
theta = [0.13; -0.004; 3.1]
lambda = 0.82
beta = 0.95;

In [533]:
# State space
x = 0:10

# Index for lambda and for investment
index_lambda = Int8[1:11 [2:11;11]] 
index_i = Int8[1:11 ones(11,1)];

In [541]:
function compute_V(theta, lambda, beta)
    dist = 100;
    iter = 0
    V = zeros(11);
    V_bar = V;
    U = [- theta[1]*x - theta[2]*x.^2 (-theta[3])*ones(11,1)]

    # Iterate the Bellman equation until convergence
    while dist>1e-20

        # Expected future values (mean over possible shocks)
        Exp_V = V[index_lambda] * [1-lambda; lambda];
        V_bar = beta * (U + Exp_V[index_i])
        V_new = log.(sum(exp.(V_bar), dims=2))

        # Check distance for convergence
        dist = max(abs.(V_new - V)...);
        iter += 1;

        # Update value function
        V = V_new 
    end
    return V, V_bar
end

V, V_bar = compute_V(theta, lambda, beta)
print("Converged after ", iter, " iterations!")

Converged after 680 iterations!

In [542]:
using Distributions

# Draw shocks
k = 100000
e = rand(Gumbel(0,1), k, 2)

# Draw states
x_t = rand(x.+1,k)

# Compute investment decisions
I = ((V_bar[x_t,:] + e) * [-1;1]) .> 0

# Compute next state
x_t1 = min.(x_t .* (I.==0) + (rand(Uniform(0,1),k).<lambda), 10)

print("we observe ", sum(I), " investment decisions in ", k, " observations")

we observe 22132 investment decisions in 100000 observations

# Estimation - lambda

- First we can estimate the value of lambda as the mean 

  $$
  \hat \lambda = \mathbb E_n \Big[ (x_{t+1}-x_t) \mid i_{t}=0 \wedge x_{t}<10 \Big]
  $$

In [552]:
import Statistics

# Estimate lambda
delta = x_t1 - x_t
lambda_hat = mean(delta[(I.==0) .& (x_t.<10)])

print("Estimated lambda: ", lambda_hat)
print("\nTrue lambda:      ", lambda)

Estimated lambda: 0.8205259511401632
True lambda:      0.82

# Estimation - theta

- Take a parameter guess $\theta_0$
- Compute the corresponding value function $V(x_t | \hat \lambda, \theta_0)$
- Compute the implied choice probabilities
- Compute the likelihood

  $$
  \mathcal{L}(\theta)=\prod_{t=1}^{T}\left(\hat{\operatorname{Pr}}\left(i=1 \mid x_{t}, \theta\right) \mathbb{1}\left(i_{t}=1\right)+\left(1-\hat{\operatorname{Pr}}\left(i=0 \mid x_{t}, \theta\right)\right) \mathbb{1}\left(i_{t}=0\right)\right)
  $$

- Repeat the above to find a minimum of the likelihood function

In [571]:
function logL(theta0, lambda_hat, beta, x_t)
    # Compute value
    V, V_bar = compute_V(theta0, lambda_hat, beta)
    
    # Implied choice probabilities
    pr_I = exp.(V_bar[:,2]) ./ (exp.(V_bar[:,1]) + exp.(V_bar[:,2]))
    
    # Likelihood
    L = sum(log.(pr_I[x_t[I.==1]])) + sum(log.(1 .- pr_I[x_t[I.==0]]))
    return L
end

print("The likelihood at the true values is ", logL(theta, lambda, beta, x_t))

The likelihood at the true values is -49246.706953102825

In [570]:
using Optim

# Initialize
theta0 = Float64[0,0,0]

# Optimize
opt = optimize((x -> -logL(x, lambda_hat, beta, x_t)), theta0)
print("Estimated thetas: ", opt.minimizer)
print("\nTrue thetas:      ", theta)

Estimated thetas: [0.12035090601820495, -0.0032215240288903956, 3.0547532223686016]
True thetas:      [0.13, -0.004, 3.1]

In [551]:
# We can also get info on the optimum
opt

 * Status: success

 * Candidate solution
    Final objective value:     4.924557e+04

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    99
    f(x) calls:    188


In [568]:
# Not all initial values are equally good
theta0 = Float64[1,1,1]

# Optimize
opt = optimize((x -> -logL(x, lambda_hat, beta, x_t)), theta0)
print("Estimated thetas: ", opt.minimizer)
print("\nTrue thetas:      ", theta)

Estimated thetas: [1.0, 1.0, 1.0]
True thetas:      [0.13, -0.004, 3.1]